In [116]:
import nbimporter
from ColumnTransformers import *
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
import joblib
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score
import json 
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
import pandas as pd 
import numpy as np 
from catboost import Pool 
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

In [164]:
def XgboostSearch(scale_pos_weight,df_X,y,metric="roc_auc"): 
    param_grid_xgb = {
        'n_estimators': [100],  
        'learning_rate': [0.001],  
        'max_depth': [3], 
        'subsample': [1.0],  
        'colsample_bytree': [1.0],  
        'gamma': [0.1], 
        'reg_lambda': [4], 
        'scale_pos_weight':[scale_pos_weight], # weight = 1 if negstive else wegiht=scale_pos_weight
        }  
    Lr=PipeLineGradient() 
    Lr.fit(df_X,y) 
    X=Lr.transform(df_X) 
    model=XGBClassifier()
    grid_search_xgb = GridSearchCV(model, param_grid_xgb, cv=5, scoring=metric, n_jobs=3)
    grid_search_xgb.fit(X,y)
    best_params=grid_search_xgb.best_params_
    with open("savedModels/XGBClassifier.json", "w") as f:
        json.dump(best_params, f, indent=4)    
    return  best_params

def LightgbmSearch(df_X,y,metric="roc_auc"):
    param_grid_lgb = {
        'n_estimators': [100],
        'learning_rate': [0.001],
        'max_depth': [3],
        'subsample': [0.7],
        'colsample_bytree': [0.6],
        'min_split_gain': [0],   # similar to gamma from XGB
        'reg_lambda': [0], 
        'is_unbalance':[True] }

    Lr=PipeLineGradient() 
    Lr.fit(df_X,y) 
    X=Lr.transform(df_X) 
    model=LGBMClassifier()
    grid_search_lgb = GridSearchCV(model, param_grid_lgb, cv=5, scoring=metric, n_jobs=3)
    grid_search_lgb.fit(X,y)
    best_params=grid_search_lgb.best_params_
    with open("savedModels/LGBMClassifier.json", "w") as f:
        json.dump(best_params, f, indent=4)
    return  best_params




def AdaSearch(df_X,y,metric='roc_auc') : 
    param_grid_ada = {
    'n_estimators': [100], 
    'learning_rate': [0.6],  # before amount of say
    'estimator': [DecisionTreeClassifier(max_depth=2)#, 
                         # DecisionTreeClassifier(max_depth=3),
                          #DecisionTreeClassifier(max_depth=5)
                 ]
    }
    Lr=PipeLineGradient() 
    Lr.fit(df_X,y) 
    X=Lr.transform(df_X) 
    model= AdaBoostClassifier()
    grid_search_ada = GridSearchCV(model, param_grid_ada, cv=5, scoring=metric, n_jobs=3)
    grid_search_ada.fit(X,y)
    best_params=grid_search_ada.best_params_
    serializable_params = {k: str(v) for k, v in best_params.items()}
    with open("savedModels/AdaBoost.json", "w") as f:
        json.dump(serializable_params, f, indent=4)
    print(f"Adaboost score: {123}")
    return best_params


    

In [194]:
def ReadCreateModel(modelName): 
    if modelName!="CatBoostClassifier":
        with open(f"savedModels/{modelName}.json", "r") as f:
            best_params=json.load(f) 
    if modelName == "AdaBoost":
            if "learning_rate" in best_params:
                best_params["learning_rate"] = float(best_params["learning_rate"])
            if "n_estimators" in best_params:
                best_params["n_estimators"] = int(best_params["n_estimators"])
            if "estimator" in best_params and isinstance(best_params["estimator"], str):
                estimator_str = best_params.pop("estimator")
                if estimator_str.startswith("DecisionTreeClassifier"):
                    estimator = eval(estimator_str) 
                else:
                    raise ValueError(f"Unsupported estimator: {estimator_str}")
                createdModel = AdaBoostClassifier(estimator=estimator)
            else:
                createdModel = AdaBoostClassifier()
    elif modelName=="XGBClassifier":
        createdModel=XGBClassifier(random_state=42, verbosity=0, n_jobs=-1)
    elif modelName=="LGBMClassifier":
        createdModel=LGBMClassifier(random_state=42 ,n_jobs=-1,verbosity=0)
    else:   
        with open(f"savedModels/{modelName}.json", "r") as f:
            data=json.load(f) 
            best_params = data["params"]
        createdModel=CatBoostClassifier(random_state=42,verbose=0)
    createdModel.set_params(**best_params) 
    return createdModel
    
def TestingModel(modelName,X_train,X_test,y_train,y_test,threshold=0.5 ): 
    
    Model=ReadCreateModel(modelName) 
    if modelName!="CatBoostClassifier":
        Model=PipelineModel(Model)
        Model.fit(X_train,y_train) 
        y_scores=Model.predict_proba(X_test)[:, 1]
    else:
        X_train_transformed,cat_train=MergeNumCat(X_train)
        X_test_transformed,cat_test=MergeNumCat(X_test)
        pool_train=Pool(data=X_train_transformed,label=y_train,cat_features=cat_train)
        pool_test = Pool(data=X_test_transformed, cat_features=cat_test)
        Model.fit(pool_train)
        y_scores= Model.predict_proba(pool_test)[:,1]
        
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    roc_auc = auc(fpr, tpr)
    aucPlot(fpr, tpr,roc_auc)

    y_pred = (np.array(y_scores) > threshold).astype(int)
    print(f"AUC: {roc_auc}, F1 score: {f1_score(y_test,t_pred)}")
    return Model


def aucPlot(fpr,tpr,roc_auc): 
    plt.figure()
    plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], linestyle="--", color="gray") 
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.grid(True)
    plt.show()

In [196]:
def CatBoostTransformer(Numerical=['Transaction.Amount', 'Customer.Age','Account.Age.Days','Quantity']): 
    column_transformer = ColumnTransformer([
        ('time_features', TimeTransformer(),["Transaction.Date","Transaction.Hour"]), 
        ("high_amount",HighAmountTransformer(),["Transaction.Amount"]),
        ("numerical",StandardScaler(),Numerical), 
        ("age",AgeTransfomer(),["Customer.Age"]), 
        ("dropColumns",'drop',["Transaction.Date","Transaction.Hour"])],remainder="passthrough") 
    return column_transformer

def MergeNumCat(X):
    CatTransformer=CatBoostTransformer() 
    CatTransformer.fit(X) 
    X_transformed=CatTransformer.transform(X)
    categorical=[col for col in range(X_transformed.shape[1]) if len(np.unique(X_transformed[:, col]))<6] 
    X_transformed[:, categorical]=X_transformed[:, categorical].astype(str)
    return X_transformed,categorical

    
def CatBoostSearch(X,y,metric="AUC"): 
    param_grid_cat = {
    'iterations': [100,200],           # Number of boosting iterations (trees)
    'learning_rate': [0.001],     # Step size shrinkage
    'depth': [4],                      # Depth of each tree
    'l2_leaf_reg': [1],                # L2 regularization
    'border_count': [32],           # Number of splits for numerical features                  
    'auto_class_weights':['Balanced'],           # Class weight scaling (for imbalanced data)
    'grow_policy': ['SymmetricTree'],  
    'colsample_bylevel': [1.0],         # Subsample ratio of columns for each split level
    'min_data_in_leaf': [1],         # Minimum samples in leaf
    'max_leaves': [31]
    }
    X_train,cat_features=MergeNumCat(X)
    
    train_pool = Pool(data=X_train, label=y, cat_features=cat_features)
    
    model = CatBoostClassifier(verbose=1,)
    
    grid_result=model.grid_search(param_grid_cat, X=train_pool, y=None,cv=5,refit=True)
    
    with open(f"savedModels/CatBoostClassifier.json", "w") as f:
        json.dump(grid_result, f, indent=4)
    return  grid_result

# def GridCatBoostSearch(params,df_X,y,metric="AUC",n_splits=5):
#     BestModel=None 
#     BestAucScore=float("-inf")
#     BestIteration=None
#     X,cat_features=MergeNumCat(df_X)
  
#     cat_features=list(cat_features)
#     X[:, cat_features]=X[:, cat_features].astype(str)

#     for param in  list(ParameterGrid(params)): 
        
#         Model=CatBoostClassifier(eval_metric=metric,early_stopping_rounds =15,random_state=42,**param)
#         SplitMatrix=kfold_split_np(X,y, n_splits)
#         aucVector=np.zeros(n_splits)
#         NumberIterations=np.zeros(n_splits) 
#         counter=0
#         for split in SplitMatrix:
#             X_train, X_test, y_train, y_test=split[0],split[1],split[2],split[3]
#             train_pool=Pool(data=X_train,label=y_train,cat_features=cat_features)
#             validation_pool = Pool(data=X_test,label=y_test,cat_features=cat_features)            
#             Model.fit(train_pool,eval_set=validation_pool)
#             y_proba = Model.predict_proba(X_test)[:, 1]
#             auc = roc_auc_score(y_test, y_proba)
#             aucVector[counter]=auc 
#             NumberIterations[counter]=Model.get_best_iteration()
#             counter+=1
#         meanAuc=np.mean(aucVector)
#         meanBestIteration=np.mean(NumberIterations)
#         if BestAucScore<meanAuc:
#             BestAucScore=meanAuc
#             BestModel=param
#             BestIteration=meanBestIteration
#     BestModelo["iterations"]=int(round(BestIterationt))
#     with open(f"savedModels/CatBoostClassifier.json", "w") as f:
#         json.dump(BestModel, f, indent=4)
#     return BestAucScore,BestModel

In [ ]:
def GiantSearch(): 
    X,y=KCrossData()
    CatBoostSearch(X,y)
    XgboostSearch(GetScalePosWeight(y),X,y)
    LightgbmSearch(X,y)
    AdaSearch(X,y) 
    print("Well done") 
    return 

In [197]:
## NOT SURE WHETER IT WORKS

def ImportanceGetter(pipeline): 
    model = pipeline.named_steps["model"]
    importances = model.feature_importances_
    preprocessor = pipeline.named_steps["preprocessing"]
    feature_names = preprocessor.get_feature_names_out()
    feature_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
    }).sort_values(by="Importance", ascending=False)
    return  feature_importance_df
